In [ ]:
#LOAD AND PROCESS MOTIFS TO CREATE AN ALL MOTIFS DATASET

motifs_15 = pd.read_csv("../data/0711_motif_manyregions_1.5std.csv").set_index(["seq","jaspar_id"]).assign(m15=1)
motifs_05 = pd.read_csv("../data/0711_motif_manyregions_.5std.csv").set_index(["seq","jaspar_id"]).assign(m05=1)
motifs_10 = pd.read_csv("../data/0711_motif_manyregions_1std.csv").set_index(["seq","jaspar_id"]).assign(m10=1)
motifs_fewer_allbest = pd.read_csv("../data/0711_motif_fewerregions_allbest.csv").set_index(["seq","jaspar_id"]).assign(fewer_allbest=1)
motifs_fewer_decent = pd.read_csv("../data/0711_motif_fewerregions_decenthits.csv").set_index(["seq","jaspar_id"]).assign(mdecent=1)
motifs_fewer_great = pd.read_csv("../data/0711_motif_fewerregions_greathits.csv").set_index(["seq","jaspar_id"]).assign(mgreat=1)
motifs_fewer_ok = pd.read_csv("../data/0711_motif_fewerregions_okhits.csv").set_index(["seq","jaspar_id"]).assign(mok=1)
motifs_allbest = pd.read_csv("../data/0711_motif_manyregions_allbest.csv").rename({"motif_id":"jaspar_id"},axis="columns").set_index(["seq","jaspar_id"]).assign(many_allbest=1)
all_motifs = pd.concat([motifs_15,motifs_05,motifs_10,motifs_fewer_allbest,motifs_allbest,motifs_fewer_decent,motifs_fewer_great,motifs_fewer_ok],axis=1)
all_motifs = all_motifs.filter(regex="^[^U]")



#DO CORRELATED TF MOTIFS SHOW UP IN REGIONS SELECTED

#link lin's correlations to the jaspar dataset
jaspar_hits_by_assay = all_motifs.groupby(level= 1).sum()
sequence_motif_counts = all_motifs.groupby(level = 0).sum()

tf_correlations = pd.read_csv("../data/20181106_APBvs7kTF_correlation.csv")

tf_cor_upper = tf_correlations.set_index(tf_correlations.Gene.str.upper()).rename({"Correlation_Coefficiency":"corr"},axis="columns")
out = jaspar.apply(lambda x: pd.Series(dict([tf_cor_upper.loc[e.upper()] for e in x["name"].split("::") if e in tf_cor_upper.index ])),axis = 1)
jaspar_correlations = out.reset_index().melt(id_vars = "jaspar_id",value_name="corr",var_name="gene").dropna().reset_index(drop =True)

jaspar_max_corrs = jaspar_correlations.groupby("jaspar_id")["corr"].max()
all_motif_corrs= all_motifs.join(jaspar_correlations.groupby("jaspar_id")["corr"].max())[["corr"]]

#WEIRD AD HOC NORMALIATION
normalized_hits = all_motifs.groupby(level=1)\
    .apply(lambda x: x/ jaspar_hits_by_assay.loc[x.name]).sum(axis=1)
normalization = normalized_hits.groupby(level=[0,1]).apply(lambda x: jaspar_max_corrs.loc[x.name[1]] if x.name[1] in jaspar_max_corrs.index else 0)
normalized_hits_by_seq =(normalized_hits * normalization).groupby(level=0).sum()



